In [1]:
import tensorflow as tf
import larq as lq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from binaryflow.layers import XnorNet
from binaryflow import quantizers
tensorflow=tf

2022-08-04 17:22:26.174028: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-08-04 17:22:26.174105: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: fedora
2022-08-04 17:22:26.174113: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: fedora
2022-08-04 17:22:26.174217: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.57.0
2022-08-04 17:22:26.174234: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.57.0
2022-08-04 17:22:26.174239: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.57.0
2022-08-04 17:22:26.174473: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operat

In [297]:
class ABCBase(tensorflow.keras.Model):
    """
    This is the base class of ABCNetworks
    It contains the functions shared by any ABCNet
    <ul>
        <li> <strong> hyperparameters </strong>:
            <ul>
                <li> kernel_estimators:  number of kernel quantizers</li>
                <li> input_estimators: number of input quantizers</li>
                <li> kernel_quantizers: the quantizers used for the kernel. defaults to ShiftedSteSign </li>
                <li> input_quantizers: the quantizers used for the input. defaults to ShiftedSteSign </li>
                <li> estimators: the estimators used to estimate the output of the non-binarised version of the layer. The content of estimators should be of the same class, and should contain the parameters of the underlying models </li>
            </ul>
        <li> <strong>input</strong>: x a tensor, with adequate shape (see subclasses) </li>
        <li> <strong>output</strong>: ABC(x)
    </ul>
    """
    def __init__(self,*args,**kwargs):
        super(ABCBase,self).__init__(*args,**kwargs)
        self.estimator=lq.layers.QuantDense(30,input_quantizer="ste_sign",kernel_quantizer="ste_sign",kernel_constraint="weight_clip")
    pass

    def build(self,input_shape):
        super(ABCBase,self).build(input_shape)
        self.estimator.build(input_shape)
        self.inputs=tf.keras.layers.Input(input_shape[1:])
        self.outputs=self.call(self.inputs)
        

    def call(self,inputs,training=False):
        return self.estimator(inputs)
    
    
        

        
        
        
estimators=[lq.layers.QuantDense(3,input_quantizer="ste_sign",kernel_quantizer="ste_sign",kernel_constraint="weight_clip") for i in range(5)]


x=tf.keras.Input(shape=(30,))
M=ABCBase()

In [298]:
M.compile(
    tf.keras.optimizers.Adam(lr=0.01, decay=0.0001),
    loss="MSE")
X=np.random.normal(0,10,[1000,30])
M.fit(X,X,epochs=5)

Epoch 1/5


/home/ramizouari/anaconda3/envs/tensorflow_2_9/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


32/32 [==============================] - 0s 1ms/step - loss: 124.1269
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 111.5900
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 112.5790
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 113.2290
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 114.1696


In [299]:
lq.models.summary(M)

+abc_base_49 stats--------------------------------------------------------------+
| Layer            Input prec.   Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs |
|                        (bit)                x 1       x 1    (kB)             |
+-------------------------------------------------------------------------------+
| quant_dense_282            1  (-1, 30)      900        30    0.23         900 |
+-------------------------------------------------------------------------------+
| Total                                       900        30    0.23         900 |
+-------------------------------------------------------------------------------+
+abc_base_49 summary-------------------------+
| Total params                      930      |
| Trainable params                  930      |
| Non-trainable params              0        |
| Model size                        232.50 B |
| Model size (8-bit FP weights)     142.50 B |
| Float-32 Equivalent               3.63 KiB |
| Compression R

In [257]:
profiler=lq.models.ModelProfile(M)

In [258]:
profiler.output_shape

In [259]:
M.layers

In [268]:
x=tf.keras.layers.Input(shape=(24))
y=tf.keras.layers.Add()([XnorNet.ScaledQuantDense(24,input_quantizer="ste_sign",kernel_quantizer="ste_sign",kernel_constraint="weight_clip")(x) for i in range(32)])


In [269]:
M=tf.keras.Model(x,y)

In [270]:
M.compile(
    tf.keras.optimizers.Adam(lr=0.01, decay=0.0001),
    loss="MSE")
X=np.random.normal(0,10,[1000,24])
M.fit(X,X,epochs=5)

Epoch 1/5
32/32 [==============================] - 4s 3ms/step - loss: 440.7860
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 117.3351
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 94.4985
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 98.2554
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 96.9802


In [271]:
lq.models.summary(M)

+model_19 stats-------------------------------------------------------------------------------------+
| Layer                   Input prec.   Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs  32-bit MACs |
|                               (bit)                x 1       x 1    (kB)                          |
+---------------------------------------------------------------------------------------------------+
| input_35                          -  (-1, 24)        0         0       0           ?            ? |
| scaled_quant_dense_250            1  (-1, 24)      576        48    0.26         576           24 |
| scaled_quant_dense_251            1  (-1, 24)      576        48    0.26         576           24 |
| scaled_quant_dense_252            1  (-1, 24)      576        48    0.26         576           24 |
| scaled_quant_dense_253            1  (-1, 24)      576        48    0.26         576           24 |
| scaled_quant_dense_254            1  (-1, 24)      576        48    0.26        

In [176]:
X

array([[  3.20782526,  10.76814446, -12.93090936, ...,  10.50177195,
        -14.42887378, -10.3534466 ],
       [  5.95250671,  11.03056017,  13.01424018, ...,   9.34781438,
         10.48834329,   4.36465717],
       [  3.43643614,   2.4846746 , -10.45888982, ...,  -2.63027835,
         -0.17769273,  21.37086089],
       ...,
       [  6.08230481, -13.28203713, -27.80959159, ...,   6.27673067,
          4.43736327, -10.40731112],
       [ -8.008216  , -13.84987933,  -3.77006756, ...,   0.19840277,
         -1.55798806, -12.06207666],
       [ -0.19244717,  -7.99895972,   4.64257551, ...,   4.08358993,
         11.33182605, -19.11890498]])

In [181]:
profiler=lq.models.ModelProfile(M)

In [182]:
profiler.op_count("mac")

0

In [183]:
isinstance(M,tf.keras.Model)

True

In [197]:
isinstance(M.layers[0],lq.layers.QuantDense)

True

In [260]:
M.summary()

Model: "abc_base_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quant_dense_207 (QuantDense  multiple                 930       
 )                                                               
                                                                 
Total params: 930
Trainable params: 930
Non-trainable params: 0
_________________________________________________________________


In [272]:
M.output

<KerasTensor: shape=(None, 24) dtype=float32 (created by layer 'add_6')>

In [261]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

model = MyModel()


In [266]:
S=MyModel()
S(tf.constant(5,shape=(10,10)))

<tf.Tensor: shape=(10, 5), dtype=float32, numpy=
array([[0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.91707414, 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.9170741 , 0.05014604],
       [0.02589533, 0.0058961 , 0.00098843, 0.9170741 , 0.05014604]],
      dtype=float32)>

In [296]:
M.output

AttributeError: Layer abc_base_48 has no inbound nodes.